# Part 3
### Pouya Farivar | 99106511

Lets get into how the models do. We start with loading the results in: Note that we saved them in the tweets dataframe in the bow_pred and roberta_pred columns.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/var/folders/cy/s7_pzg4x2xxcg7cty448hrxc0000gn/T/ipykernel_10487/2080034654.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


reading the datframes. note that the tweets is loaded from tweets_pred and it has the two best model predictions.

In [51]:
companies = pd.read_csv('companies.csv')
entities = pd.read_csv('entities.csv')
tweets = pd.read_csv('tweets_pred.csv')
users = pd.read_csv('users.csv', on_bad_lines='skip')

/var/folders/cy/s7_pzg4x2xxcg7cty448hrxc0000gn/T/ipykernel_10487/3197010514.py:4: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('users.csv', on_bad_lines='skip')


Okay now this is the most important part. We are supposed to choose a time line but the time line of the data itself is around 11 month which is not anything crazy in order to achieve a conclusion on whether there is any correlation between the number of tweets and capitalization. So i have used the entire duration of the tweets and here i group them based on number of views and how positive or negative the reviews are:

In [55]:
tweets['bow_pred'] = tweets['bow_pred'].replace(0, -1)
merged_data = pd.merge(entities_symbol[['tweet_id', 'text']], tweets, left_on='tweet_id', right_on='id')

# getting the average prediction
average_sentiment = merged_data.groupby('text_x')['bow_pred'].mean().reset_index()
average_sentiment.rename(columns={'bow_pred': 'average_bow_pred'}, inplace=True)
companies = pd.merge(companies, average_sentiment, left_on='ticker', right_on='text_x', how='left')
companies['average_bow_pred'].fillna(-1, inplace=True)  # Replace NaN values with -1 if CashTag has no associated tweets

# getting the number of tweets overall
tweets_count = entities_symbol['text'].value_counts().reset_index()
tweets_count.columns = ['ticker', 'tweets_count']
companies = pd.merge(companies, tweets_count, on='ticker', how='left')
companies['tweets_count'].fillna(0, inplace=True)

useful_companies = companies[(companies['average_bow_pred'] != 1) & (companies['average_bow_pred'] != -1)]
useful_companies = useful_companies[(useful_companies['tweets_count'] != 0)]
useful_companies = useful_companies.dropna()


/var/folders/cy/s7_pzg4x2xxcg7cty448hrxc0000gn/T/ipykernel_10487/1920612641.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  companies['average_bow_pred'].fillna(-1, inplace=True)  # Replace NaN values with -1 if CashTag has no associated tweets
/var/folders/cy/s7_pzg4x2xxcg7cty448hrxc0000gn/T/ipykernel_10487/1920612641.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never

Now lets see if there is any correlation between market capitalization and the postitivity of the tweets:

In [50]:
from scipy.stats import spearmanr, pearsonr

# Spearman correlation
spearman_corr, _ = spearmanr(useful_companies['capitalization'].values/np.max(useful_companies['capitalization'].values), useful_companies['average_bow_pred'].values)

# Pearson correlation
pearson_corr, _ = pearsonr(useful_companies['capitalization'].values/np.max(useful_companies['capitalization'].values), useful_companies['average_bow_pred'].values)

print(f"Spearman Correlation: {spearman_corr}")
print(f"Pearson Correlation: {pearson_corr}")

Spearman Correlation: 0.016079915153166615
Pearson Correlation: 0.05295597795161083


No, No there is not. In economical and financial world there is no such thing as real correlations. We attempted to find one here and we saw that the results, close to zero correlation coefs, show that there is no such thing as correlation between capitalization and positiveness of the tweets.

I will also add that this probably is because of the fact that the market is a 2nd degree complex system. This means that the information about the system changes the state of the system. You would think that there should be a correlation between the two but the causal and caused realtionship between these two are so complec that a simple correlation coef doesnt provide any alpha.

In [57]:
from scipy.stats import spearmanr, pearsonr

# Calculate Spearman correlation
spearman_corr, _ = spearmanr(useful_companies['tweets_count'].values/np.max(useful_companies['tweets_count'].values), useful_companies['average_bow_pred'].values)

# Calculate Pearson correlation
pearson_corr, _ = pearsonr(useful_companies['tweets_count'].values/np.max(useful_companies['tweets_count'].values), useful_companies['average_bow_pred'].values)

print(f"Spearman Correlation: {spearman_corr}")
print(f"Pearson Correlation: {pearson_corr}")

Spearman Correlation: 0.1480047008670637
Pearson Correlation: 0.0650992476636284


This is for the correlation between tweet counts and the state of the overall tweets. Here again you would excpect that there would be a strong correlation. This may be because of the fact that our data is not clean enogh or just enough to prove this, but you can see that the correlation measures are bigger than last time, idicationg that there is infact a correlation between the two.

Most of the time an increase in the tweet activity means volitility in the markets which is bad for the price action and can cause a more negative tweet sentiment. which can be understood intuituvely.

We can also say that the poor results could be becuase of the fact that we only achived 78% accuracy on Part 2. This error could cause the real correlation to hide beneth the preditions but i really dount that. Since we are working with averages and that cancels alot of the noise and error.

### Transfer learning

Finally lets talk about transfer learning because we used it for both the bert and alberta models. These models are huge models and thus using transfer learning we were able to freeze most of the models as we saw in the last part and only train the last layers which we needed. This reduces the training time and the resources needed alot and it also makes the loaded models stronger on our data as well. I belive that transfer learning is one of the best ways to jump start anything and then move to making on yourself.

Thats all. I tried to keep it short and a 100.

### Bonus
Also i have scrapped some dat from digicala for the Bonus part check it out.